In [10]:
import tensorflow as tf
import cv2
import numpy as np
import pandas as pd
import pydicom

In [11]:
#Load trained model
model = tf.keras.models.load_model("./weights/resnet_covid.hdf5")

In [12]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [13]:
# #Individual DICOM Preprocessing code
# def preprocess(path):
#     ds = pydicom.read_file(path)
#     shape = ds.pixel_array.shape
#     # # Convert to float to avoid overflow or underflow losses.
#     image_2d = ds.pixel_array.astype(float)
#     #
#     # # Rescaling grey scale between 0-255
#     image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0
#     #
#     # # Convert to uint
#     image_2d_scaled = np.uint8(image_2d_scaled)
#     img = cv2.resize(image_2d_scaled, (224, 224))
#     img = np.dstack((img, img, img))
#     img = np.expand_dims(img, 0)
#     return img
    
    



In [14]:
# #Predictions using individual preprocessed dicoms
# preds = []
# for path in df["Path"]:
#     preds.append((model.predict(preprocess(path))))

In [15]:
#If images are extracted and converted to pngs using the above preprocessing...
#Compile data into a dataframe
df = pd.read_csv("example.csv")
df["Label"] = df["Label"].astype(str)
df.head()

,Unnamed: 0,Path,Label
0,0,./pos_example.png,1
1,1,./neg_example.png,0


In [16]:
#Preprocessing is done in these data loaders by rescaling the pixel values between 0 and 255 as well as resizng the input 
#to 224x224 images
eval_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255
)
inference = eval_gen.flow_from_dataframe(
    df,
    x_col="Path",
    y_col="Label",
    target_size=(224, 224),
    class_mode="binary",
    batch_size=2,
)



Found 2 validated image filenames belonging to 2 classes.


In [17]:
pred = model.predict(inference, len(inference.filenames))

In [18]:
print(pred)

[[0.31518343]
 [0.9228561 ]]
